### Launch Sites Locations Analysis with Folium

In [1]:
import folium
import pandas as pdimport pandas as pdimport pandas as pdimport pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [14]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(URL)
#spacex_df.head()

In [104]:
## take a look at what are the coordinates for each site

spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# first () returns the first row it encounters in the original dataset for each unique value in the specified grouping column
launch_site = spacex_df.groupby(['Launch Site'], as_index= False).first() 
launch_site = launch_site[['Launch Site', 'Lat','Long']]
launch_site

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [76]:
## We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)


In [77]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [79]:
## add a circle for each launch site in data frame launch_sites
# using tolist() creates a list containing the latitude and longitude values of the first launch site

first_launchsite = launch_site[['Lat', 'Long']].iloc[0].tolist()
site_map = folium.Map(location=first_launchsite, zoom_start=10)
label = launch_site[['Launch Site']].iloc[0].tolist()

In [93]:
## I do not want to make the same for all the launch sites, so create a function that takes coordinates and names and plot them
## automatically. Reduced the zoom to see them all 

def plot_launch_sites(launch_site):
    site_map = folium.Map(location=[launch_site['Lat'].iloc[0], launch_site['Long'].iloc[0]], zoom_start=4)

    for i in range(len(launch_site)):
        site_name = launch_site['Launch Site'].iloc[i]
        site_coordinates = [launch_site['Lat'].iloc[i], launch_site['Long'].iloc[i]]

        # Circle marker
        circle = folium.Circle(location=site_coordinates, radius=1000, color='#000000', fill=True).add_child(folium.Popup(site_name))
        site_map.add_child(circle)

        # Custom marker with label
        custom_marker = folium.Marker(location=site_coordinates, icon=folium.DivIcon(icon_size=(20,20), icon_anchor=(0,0), html=f'<div style="font-size: 12; color:#d35400;"><b>{site_name}</b></div>'))
        site_map.add_child(custom_marker)

    return site_map

# Assuming 'launch_site' DataFrame is the result of the previous operation
launch_site_map = plot_launch_sites(launch_site)

display(launch_site_map)


In [103]:
## enhance the map by adding the launch outcomes for each site
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
spacex_df.tail(10)


,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [144]:
## If a launch was successful (class=1), then we use a green marker and if a launch was failed, we use a red marker (class=0)
import numpy as np

marker_cluster = MarkerCluster()

spacex_df['marker color'] = np.where(spacex_df['class'] == 1, 'green', 'red')
spacex_df.tail()

,Launch Site,Lat,Long,class,marker color
51,CCAFS SLC-40,28.563197,-80.57682,0,red
52,CCAFS SLC-40,28.563197,-80.57682,0,red
53,CCAFS SLC-40,28.563197,-80.57682,0,red
54,CCAFS SLC-40,28.563197,-80.57682,1,green
55,CCAFS SLC-40,28.563197,-80.57682,0,red


In [143]:
spacex_df['marker color'] = spacex_df['marker color'].astype('object')

In [146]:
site_map.add_child(marker_cluster)

In [148]:
for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker color']
    marker_icon = folium.Icon(color='white', icon_color=marker_color)
    marker = folium.Marker(coordinate, icon=marker_icon)
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [149]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [150]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [151]:
# Given coordinates
launch_site_lat = 28.56367
launch_site_lon = -80.57163
coastline_lat = 28.56342  # provide the latitude of the coastline
coastline_lon = -80.56744  # provide the longitude of the coastline

# Calculating distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(f"The distance to the closest coastline is {distance_coastline} kilometers.")

The distance to the closest coastline is 0.410271918104809 kilometers.


In [160]:
# Create a map centered on the launch site
m = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Create a marker for the launch site
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    icon=folium.Icon(color='red', icon='rocket', prefix='fa')
).add_to(m)

# Create a marker for the coastline point and add the distance
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
distance_marker.add_to(m)

# Display the map
m

In [159]:
# Create a PolyLine object with the coastline and launch site coordinates
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=1
)

# Add the PolyLine to the map
lines.add_to(m)

m

In [167]:
import folium

# Given coordinates
launch_site_lat = 28.56341
launch_site_lon = -80.57678
railway_lat = 28.57205
railway_lon = -80.58527

# Calculating distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# Create a map centered on the launch site
m = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Create a PolyLine object with the coastline and launch site coordinates
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]],
    weight=1
)

# Add the PolyLine to the map
lines.add_to(m)

# Create a marker for the coastline point and add the distance
distance_marker = folium.Marker(
    location=[railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
distance_marker.add_to(m)


marker_cluster = MarkerCluster()

# Create a function to assign marker colors based on the value of the class column
def assign_marker_color(row):
    if row['class'] == 1:
        return 'green'
    elif row['class'] == 0:
        return 'red'

# Apply the function to create the marker_color column
spacex_df['marker_color'] = spacex_df.apply(assign_marker_color, axis=1)



m.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    marker_icon = folium.Icon(color='white', icon_color=marker_color)
    marker = folium.Marker(coordinate, icon=marker_icon)
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)
    

# Iterate over each launch site
for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']
    
    # Add a Circle object for the launch site with a popup label
    circle = folium.Circle(coordinate, radius=50, color='#d35400', fill=True).add_child(folium.Popup(launch_site_name))
    m.add_child(circle)
    
    # Add a Marker object for the launch site with a popup label
    marker = folium.Marker(
        coordinate,
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % launch_site_name,
        )
    )
    marker.add_child(folium.Popup(launch_site_name))
    m.add_child(marker)

# Display the map
m

In [168]:
import folium

# Given coordinates
launch_site_lat = 28.56341
launch_site_lon = -80.57678
highway_lat = 28.56339
highway_lon = -80.57077

# Calculating distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Create a map centered on the launch site
m = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Create a PolyLine object with the coastline and launch site coordinates
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]],
    weight=1
)

# Add the PolyLine to the map
lines.add_to(m)

# Create a marker for the coastline point and add the distance
distance_marker = folium.Marker(
    location=[highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
distance_marker.add_to(m)


marker_cluster = MarkerCluster()

# Create a function to assign marker colors based on the value of the class column
def assign_marker_color(row):
    if row['class'] == 1:
        return 'green'
    elif row['class'] == 0:
        return 'red'

# Apply the function to create the marker_color column
spacex_df['marker_color'] = spacex_df.apply(assign_marker_color, axis=1)



m.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    marker_icon = folium.Icon(color='white', icon_color=marker_color)
    marker = folium.Marker(coordinate, icon=marker_icon)
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)
    

# Iterate over each launch site
for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']
    
    # Add a Circle object for the launch site with a popup label
    circle = folium.Circle(coordinate, radius=50, color='#d35400', fill=True).add_child(folium.Popup(launch_site_name))
    m.add_child(circle)
    
    # Add a Marker object for the launch site with a popup label
    marker = folium.Marker(
        coordinate,
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % launch_site_name,
        )
    )
    marker.add_child(folium.Popup(launch_site_name))
    m.add_child(marker)

# Display the map
m

In [169]:
import folium

# Given coordinates
launch_site_lat = 28.56341
launch_site_lon = -80.57678
city_lat = 28.40167
city_lon = -80.60394

# Calculating distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Create a map centered on the launch site
m = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Create a PolyLine object with the coastline and launch site coordinates
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]],
    weight=1
)

# Add the PolyLine to the map
lines.add_to(m)

# Create a marker for the coastline point and add the distance
distance_marker = folium.Marker(
    location=[city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
distance_marker.add_to(m)


marker_cluster = MarkerCluster()

# Create a function to assign marker colors based on the value of the class column
def assign_marker_color(row):
    if row['class'] == 1:
        return 'green'
    elif row['class'] == 0:
        return 'red'

# Apply the function to create the marker_color column
spacex_df['marker_color'] = spacex_df.apply(assign_marker_color, axis=1)



m.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    marker_icon = folium.Icon(color='white', icon_color=marker_color)
    marker = folium.Marker(coordinate, icon=marker_icon)
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)
    

# Iterate over each launch site
for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']
    
    # Add a Circle object for the launch site with a popup label
    circle = folium.Circle(coordinate, radius=50, color='#d35400', fill=True).add_child(folium.Popup(launch_site_name))
    m.add_child(circle)
    
    # Add a Marker object for the launch site with a popup label
    marker = folium.Marker(
        coordinate,
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % launch_site_name,
        )
    )
    marker.add_child(folium.Popup(launch_site_name))
    m.add_child(marker)

# Display the map
m